In [1]:
import matplotlib.pyplot as plt
import random
import collections
from matplotlib.colors import ListedColormap
from sklearn import datasets
import numpy as np

In [2]:
# класс узла

class Node:
    
    def __init__(self, index, t, true_branch, false_branch):
        self.index = index  # индекс признака, по которому ведется сравнение с порогом в этом узле
        self.t = t  # значение порога
        self.true_branch = true_branch  # поддерево, удовлетворяющее условию в узле
        self.false_branch = false_branch  # поддерево, не удовлетворяющее условию в узле

In [3]:
# класс терминального узла (листа)

class Leaf:
    
    def __init__(self, data, labels):
        self.data = data
        self.labels = labels
        self.prediction = self.predict()
        
    def predict(self):
   
        prediction = np.mean(self.labels)
        return prediction    

In [4]:
class DecisionTreeRegressor_alg:
    def __init__(
        self,
        max_depth,
        min_samples_leaf,
        random_state): #здесь не нужен, для приведения к одному виду с DecisionTreeRegressor
        
        self.max_depth = max_depth
        self.min_samples_leaf = min_samples_leaf
        
        self.inf_criteria = self.calc_var #if crit == 'var' else self.calc_gini

        self.tree = None
        

    def fit(self, X, y):
        self.tree = self.build_tree(X, y)
        return self
    
    
    def predict(self, X):
    
        classes = []
        for obj in X:
            prediction = self.classify_object(obj, self.tree)
            classes.append(prediction)
        return classes
    
    
    @staticmethod
    def calc_var(y_pred):

        return y_pred.std()**2
   

    def quality(self, left_labels, right_labels, current_criteria):

        # доля выбоки, ушедшая в левое поддерево
        p = float(left_labels.shape[0]) / (left_labels.shape[0] + right_labels.shape[0])

        return current_criteria - p * self.inf_criteria(left_labels) - (1 - p) * self.inf_criteria(right_labels)
    
    
    def split(self, X, y, index, t):

        left = np.where(X[:, index] <= t)
        right = np.where(X[:, index] > t)

        true_data = X[left]
        false_data = X[right]
        true_labels = y[left]
        false_labels = y[right]

        return true_data, false_data, true_labels, false_labels
    
    
    def find_best_split(self, X, y): 

        current_criteria = self.inf_criteria(y)
        best_quality = 0
        best_t = None
        best_index = None

        n_features = X.shape[1]

        for index in range(n_features):
            # будем проверять только уникальные значения признака, исключая повторения
            t_values = np.unique([row[index] for row in X])

            for t in t_values:
                true_data, false_data, true_labels, false_labels = self.split(X, y, index, t)
                #  пропускаем разбиения, в которых в узле остается менее min_leaf объектов
                if len(true_data) < self.min_samples_leaf or len(false_data) < self.min_samples_leaf:
                    continue

                current_quality = self.quality(true_labels, false_labels, current_criteria)

                #  выбираем порог, на котором получается максимальный прирост качества
                if current_quality > best_quality:
                    best_quality, best_t, best_index = current_quality, t, index

        return best_quality, best_t, best_index
    
    
    def build_tree(self, X, y, depth = 0):

        if depth >= self.max_depth: # критерий остановки по максимальной глубине дерева
            return Leaf(X, y)


        quality, t, index = self.find_best_split(X, y)

        #  Базовый случай - прекращаем рекурсию, когда нет прироста в качества
        if quality == 0:
            return Leaf(X, y)

        true_data, false_data, true_labels, false_labels = self.split(X, y, index, t)
        depth +=1
        
        # Рекурсивно строим два поддерева
        true_branch = self.build_tree(true_data, true_labels, depth)
        false_branch = self.build_tree(false_data, false_labels, depth)

        # Возвращаем класс узла со всеми поддеревьями, то есть целого дерева
        return Node(index, t, true_branch, false_branch)

       
    def classify_object(self, obj, node):

        #  Останавливаем рекурсию, если достигли листа
        if isinstance(node, Leaf):
            answer = node.prediction
            return answer

        if obj[node.index] <= node.t:
            return self.classify_object(obj, node.true_branch)
        else:
            return self.classify_object(obj, node.false_branch)
        

In [5]:
def calc_mse(y, y_pred):
    err = np.mean((y - y_pred)**2)
    return err

In [6]:
# сгенерируем данные
classification_data, classification_labels = datasets.make_regression(n_samples=1000, 
                                              n_features = 2, 
                                              n_informative = 2, 
                                              n_targets = 1, 
                                              noise = 5, 
#                                               coef = True, 
                                              random_state = 2)

In [7]:
# Разобьем выборку на обучающую и тестовую

from sklearn import model_selection

train_data, test_data, train_labels, test_labels = model_selection.train_test_split(classification_data, 
                                                                                     classification_labels, 
                                                                                     test_size = 0.3,
                                                                                     random_state = 1)

In [8]:
tree = DecisionTreeRegressor_alg(max_depth = 5, min_samples_leaf=5, random_state=42)
tree.fit(train_data, train_labels)

In [9]:
train_answ = tree.predict(train_data)
test_answ = tree.predict(test_data)

In [10]:
calc_mse(train_labels, train_answ), calc_mse(test_labels, test_answ)

(463.75692042517096, 721.2269271196762)

In [11]:
from sklearn.metrics import r2_score
r2_score(train_labels, train_answ), r2_score(test_labels, test_answ)

(0.9465266944201165, 0.8969516588020108)